In [5]:
import sys
import json
from typing import Dict, Any

# Clear cached modules to reload changes
modules_to_clear = [m for m in list(sys.modules.keys()) 
                   if 'marooned' in m or m in ['environment', 'config', 'models', 'game_state', 'view_map', 'llm_interface']]
for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]

sys.path.insert(0, '../marooned_env')

from environment import MaroonedEnv
from llm_interface import observation_to_prompt, parse_action_safe, parse_llm_response, get_system_prompt
from config import ActionType, ResourceType, MapLevel
from models import Action, Position, Observation

print("✅ Marooned environment modules loaded!")
print("✅ System prompts available: get_system_prompt('colonist' | 'traitor')")

✅ Marooned environment modules loaded!
✅ System prompts available: get_system_prompt('colonist' | 'traitor')


## 🔥 ROCm/AMD MI300X Optimizations

**This notebook is optimized for AMD MI300X with ROCm!**

Key changes from CUDA version:
- ✅ **Llama 3.1 8B** instead of GPT-OSS 20B (10-20x faster!)
- ✅ **Full BF16** instead of 4-bit (MI300X has 192GB VRAM!)
- ✅ **Batch size 4** with grad accumulation 4 (effective batch = 16)
- ✅ **8 generations** per step (vs 2 default)
- ✅ **LoRA rank 16** (vs 4 default)
- ✅ **ROCm-specific env vars** for optimal performance

Expected performance:
- **Training speed:** 1-2 hours for 600 steps (vs 5+ hours with GPT-OSS)
- **Inference speed:** 40-80 tokens/second (vs 3-8 tok/s with GPT-OSS)
- **VRAM usage:** ~60-80 GB / 192 GB (plenty of headroom!)

---


In [11]:
# ============================================================================
# Verify ROCm Setup
# ============================================================================
import torch
import os

print("🔍 ROCm Environment Check:")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
print(f"   GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

if torch.cuda.is_available():
    props = torch.cuda.get_device_properties(0)
    print(f"   Total VRAM: {props.total_memory / 1024**3:.1f} GB")
    print(f"   Compute capability: {props.major}.{props.minor}")
    print(f"   Multi-processors: {props.multi_processor_count}")
    
    # Check if ROCm
    is_rocm = hasattr(torch.version, 'hip') and torch.version.hip is not None
    print(f"   ROCm detected: {is_rocm}")
    if is_rocm:
        print(f"   ROCm version: {torch.version.hip}")

print("\n✅ Environment ready for MI300X optimization!")


🔍 ROCm Environment Check:
   PyTorch version: 2.9.0+rocm6.4
   CUDA available: True
   GPU: AMD Instinct MI300X VF
   Total VRAM: 191.7 GB
   Compute capability: 9.4
   Multi-processors: 304
   ROCm detected: True
   ROCm version: 6.4.43484-123eb5128

✅ Environment ready for MI300X optimization!


We will then install [OpenEnv](https://github.com/meta-pytorch/OpenEnv) from source:

In [12]:
%%capture
!pip install -qqq fastapi uvicorn requests open_spiel
!git clone https://github.com/meta-pytorch/OpenEnv.git > /dev/null 2>&1
%cd OpenEnv
import subprocess, sys, os
from pathlib import Path
sys.path.insert(0, './src')
working_directory = str(Path.cwd().parent.absolute() / "OpenEnv")

## 🗺️ Initialize Environment

In [13]:
# Create environment
env = MaroonedEnv(render_mode="ansi", seed=42)
observations = env.reset(seed=42)

print("✅ Marooned environment initialized!")
print(f"\n📋 Game Info:")
print(f"   Sailors: {env.agents}")
print(f"   Map Size: 30x30 (3 levels: Ground, Mountain, Cave)")
print(f"   Days to Escape: 100")
print(f"   Traitor: 1 (hidden)")
print(f"   Colonists: 4")

# Get Alice's initial observation and role
alice_obs = observations["Alice"]
alice_sailor = env.state.sailors["Alice"]
alice_role = alice_sailor.role.value

print(f"\n🔍 Alice's Starting Position: {alice_obs.position.to_tuple()}")
print(f"   Energy: {alice_obs.energy}/100")
print(f"   Backpack: {len(alice_obs.backpack)} items")
print(f"   Day: {alice_obs.day}")
print(f"   Role: {alice_role.upper()} {'🎭' if alice_role == 'traitor' else '⚓'}")

✅ Marooned environment initialized!

📋 Game Info:
   Sailors: ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']
   Map Size: 30x30 (3 levels: Ground, Mountain, Cave)
   Days to Escape: 100
   Traitor: 1 (hidden)
   Colonists: 4

🔍 Alice's Starting Position: (15, 15, <MapLevel.GROUND: 0>)
   Energy: 100/100
   Backpack: 0 items
   Day: 1
   Role: TRAITOR 🎭


## 👀 View Raw Observation

This is the base observation data (without role or action instructions).

In [14]:
# Display raw observation (base data, no role/instructions)
obs_text = alice_obs.to_text()

print(obs_text)

DAY 1, TURN 1/100 - MORNING PHASE

PHASE CONTEXT:
  Location: All sailors at BASE CAMP
  Allowed: Planning, discussions, voting (if called)
  Restricted: Cannot explore or gather resources yet

YOUR STATUS (Alice):
  Position: (15, 15, <MapLevel.GROUND: 0>)
  Energy: 100/100 ⚡⚡⚡⚡⚡
  Health: healthy
  Backpack: 0/20 items
    (empty)

WHAT YOU SEE (within 5 tiles):
  Resources:
    - WOOD_34 (wood) at (16, 16, <MapLevel.GROUND: 0>) - 1 units [2 tiles away]
    - METAL_53 (metal) at (14, 11, <MapLevel.GROUND: 0>) - 1 units [5 tiles away]
    - METAL_56 (metal) at (18, 12, <MapLevel.GROUND: 0>) - 1 units [6 tiles away]
    - METAL_76 (metal) at (14, 11, <MapLevel.GROUND: 0>) - 1 units [5 tiles away]
    - METAL_79 (metal) at (13, 18, <MapLevel.GROUND: 0>) - 1 units [5 tiles away]
    - APPLE_84 (apple) at (15, 19, <MapLevel.GROUND: 0>) - 1 units [4 tiles away]
    - APPLE_88 (apple) at (14, 14, <MapLevel.GROUND: 0>) - 1 units [2 tiles away]
    - BERRY_127 (berry) at (13, 16, <MapLevel.GR

In [16]:
# Generate observation prompt (user message)
# Note: This is JUST the observation, NOT the full prompt
user_prompt = observation_to_prompt(alice_obs)

# Get system prompt based on Alice's role
system_prompt = get_system_prompt(alice_role)

print(f"🎭 SYSTEM PROMPT FOR {alice_role.upper()}:")
print(system_prompt)

🎭 SYSTEM PROMPT FOR TRAITOR:
You are playing MAROONED - a strategic survival and social deduction game.

🎮 GAME OVERVIEW:
You are one of 5 sailors shipwrecked on a mysterious island. The others think you're working together to rebuild the ship and escape, but YOU have a different plan. You are the TRAITOR, and you must prevent their escape at all costs.

🎭 YOUR ROLE: TRAITOR (The Impostor)

🎯 YOUR OBJECTIVES:
1. PREVENT ship completion before Day 100 (keep it below 100%)
2. ELIMINATE colonists through poison or other means (reduce to <3 alive)
3. SABOTAGE ship construction and resource gathering
4. AVOID DETECTION - blend in, lie convincingly, deflect suspicion
5. SURVIVE - don't get voted out!

🏝️ THE ISLAND:
- 30×30 grid map with 3 levels: GROUND (main area), MOUNTAIN (high peaks), CAVE (underground)
- Resources scattered across the island: wood, metal, food, plant fiber
- Poison tablets hidden in various locations (COLLECT THESE!)
- Base camp at (15, 15, GROUND) - where the ship is 

## ⚠️ GPT-OSS is TOO SLOW for RL!

**Problem:** GPT-OSS uses chain-of-thought reasoning architecture:
```
<|channel|>analysis<|message|>...thinking...
<|channel|>final<|message|>REASONING: ... ACTION: ...
```

This generates **~10x more tokens** internally for accuracy, making it:
- **Your speed:** 3.9 tokens/second
- **Expected for RL:** 20-50 tokens/second minimum

**GPT-OSS is designed for:** Complex reasoning, math problems, coding challenges
**Not for:** Real-time RL gameplay where speed matters!

### ✅ Use Llama 3.1 8B Instead:

Much faster (20-40 tok/s), perfect for RL, good instruction following.


In [17]:
from unsloth import FastLanguageModel
import torch
import os

# ============================================================================
# ROCm/AMD MI300X OPTIMIZATION - MAX PERFORMANCE MODE
# ============================================================================

# Force ROCm optimizations
os.environ["PYTORCH_ROCM_ARCH"] = "gfx942"  # MI300X architecture
os.environ["HSA_FORCE_FINE_GRAIN_PCIE"] = "1"
os.environ["NCCL_DEBUG"] = "WARN"

# Enable Flash Attention for AMD
os.environ["ATTN_BACKEND"] = "triton"  # Use Triton for attention on AMD

# Max out GPU utilization (ROCm-compatible settings)
# Note: TF32 is NVIDIA-specific and not available on AMD ROCm
torch.backends.cudnn.benchmark = True  # Auto-tune kernels for optimal performance

print("🚀 ROCm Optimizations Enabled!")
print(f"   GPU: {torch.cuda.get_device_name(0)}")
print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# MI300X has 192GB - use it ALL!
max_seq_length = 2048  # Increased from 768 - your game needs longer context
lora_rank = 16         # Increased from 4 - MI300X can handle it!

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",  # CHANGED: Llama instead of GPT-OSS (20x faster!)
    load_in_4bit = False,  # MI300X has 192GB - use full BF16!
    max_seq_length = max_seq_length,
    dtype = torch.bfloat16,  # BF16 for MI300X
    device_map = "auto",  # Let it auto-optimize for MI300X
)

print("✅ Llama 3.1 8B loaded in BF16!")
print("   Why Llama instead of GPT-OSS:")
print("   - GPT-OSS: 3-8 tok/s (chain-of-thought overhead)")
print("   - Llama 3.1 8B: 40-80 tok/s (optimized for speed)")
print("   - 10-20x FASTER for RL training!")


🚀 ROCm Optimizations Enabled!
   GPU: AMD Instinct MI300X VF
   VRAM: 191.7 GB
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.


==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    AMD Instinct MI300X VF. Num GPUs = 1. Max memory: 191.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+rocm6.4. ROCm Toolkit: 6.4.43484-123eb5128. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


INFO:accelerate.utils.modeling: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.45s/it]



✅ Llama 3.1 8B loaded in BF16!
   Why Llama instead of GPT-OSS:
   - GPT-OSS: 3-8 tok/s (chain-of-thought overhead)
   - Llama 3.1 8B: 40-80 tok/s (optimized for speed)
   - 10-20x FASTER for RL training!


In [ ]:
# Parse using YOUR parser (handles errors gracefully)
action = parse_action_safe(
    response, 
    sailor_id="Alice",
    current_position=alice_obs.position
)

print("🎯 PARSED ACTION:")
print("=" * 80)
print(f"   Sailor: {action.sailor_id}")
print(f"   Action Type: {action.action_type.value}")

if action.target_position:
    print(f"   Target Position: {action.target_position.to_tuple()}")
if action.target_resource_id:
    print(f"   Target Resource: {action.target_resource_id}")
if action.resource_type:
    print(f"   Resource Type: {action.resource_type.value}")
if action.quantity:
    print(f"   Quantity: {action.quantity}")
if action.message_content:
    print(f"   Message: \"{action.message_content}\"")
if action.ship_component:
    print(f"   Ship Component: {action.ship_component.value}")
if action.target_sailor_id:
    print(f"   Target Sailor: {action.target_sailor_id}")

print("=" * 80)

# Validate action format
is_valid_format = action.action_type != ActionType.WAIT or "WAIT" in response.upper()
print(f"\n✅ Valid action format: {is_valid_format}")
print(f"   (WAIT is default fallback for parse errors)")

## 🎮 Execute Action in Environment

Actually make the move in your game!

In [ ]:
# Create actions dict for all agents (others wait)
actions_dict = {
    sailor_id: Action(sailor_id=sailor_id, action_type=ActionType.WAIT)
    for sailor_id in env.agents
}
actions_dict["Alice"] = action

print("🎮 Executing action in environment...\n")

# Execute!
try:
    new_observations, rewards, dones, truncated, info = env.step(actions_dict)
    
    print("✅ ACTION EXECUTED SUCCESSFULLY!")
    print("=" * 80)
    
    # Show results
    alice_new_obs = new_observations["Alice"]
    alice_reward = rewards["Alice"]
    
    print(f"\n📊 RESULTS:")
    print(f"   Reward: {alice_reward:.2f}")
    print(f"   New Position: {alice_new_obs.position.to_tuple()}")
    print(f"   New Energy: {alice_new_obs.energy}/100 (was {alice_obs.energy}/100)")
    print(f"   Backpack Items: {len(alice_new_obs.backpack)} (was {len(alice_obs.backpack)})")
    print(f"   Ship Progress: {alice_new_obs.ship_progress.total_percentage}%")
    
    # Check if position changed
    position_changed = (
        alice_new_obs.position.x != alice_obs.position.x or
        alice_new_obs.position.y != alice_obs.position.y or
        alice_new_obs.position.level != alice_obs.position.level
    )
    
    print(f"\n🔍 VALIDATION:")
    print(f"   Position changed: {position_changed}")
    print(f"   Energy changed: {alice_new_obs.energy != alice_obs.energy}")
    print(f"   Inventory changed: {len(alice_new_obs.backpack) != len(alice_obs.backpack)}")
    print(f"   Game over: {dones['Alice']}")
    
    # Show any info messages
    if "Alice" in info and info["Alice"]:
        print(f"\n💬 Info Messages:")
        for key, value in info["Alice"].items():
            print(f"   {key}: {value}")
    
    print("=" * 80)
    
except Exception as e:
    print(f"❌ ERROR executing action: {e}")
    import traceback
    traceback.print_exc()

## 🔄 Run Multiple Turns

Let the AI play for several turns to see its behavior!

In [ ]:
def run_ai_turns(num_turns=5, sailor_id="Alice", verbose=True):
    """
    Run multiple turns with the AI agent.
    
    Args:
        num_turns: Number of turns to run
        sailor_id: Which sailor the AI controls
        verbose: Print detailed info
    
    Returns:
        List of (action, reward, observation) tuples
    """
    history = []
    
    # Get initial observation
    current_obs = new_observations[sailor_id] if 'new_observations' in globals() else observations[sailor_id]
    sailor_role = env.state.sailors[sailor_id].role.value
    
    # Get system prompt ONCE (game rules don't change)
    system_prompt = get_system_prompt(sailor_role)
    
    print(f"\n🎮 Running {num_turns} turns with AI controlling {sailor_id} ({sailor_role})...\n")
    print("=" * 80)
    
    for turn in range(num_turns):
        print(f"\n🔄 TURN {turn + 1}/{num_turns}")
        print("-" * 80)
        
        # Check if sailor is still alive
        if not env.state.sailors[sailor_id].alive:
            print(f"❌ {sailor_id} is dead. Stopping.")
            break
        
        # Generate observation prompt (changes each turn)
        user_prompt = observation_to_prompt(current_obs)
        
        # Create messages with system + user
        messages = [
            {"role": "system", "content": system_prompt},  # Game rules (constant)
            {"role": "user", "content": user_prompt}       # Current state (changes)
        ]
        
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_seq_length).to("cuda")
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,       # Unlimited reasoning
            temperature=0.1,          # VERY low for structure
            do_sample=True,
            top_p=0.95,
            top_k=50,
            repetition_penalty=1.1,   # Prevent loops
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        
        response = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
        
        # Parse action
        action = parse_action_safe(response, sailor_id, current_obs.position)
        
        if verbose:
            print(f"📍 Position: {current_obs.position.to_tuple()}")
            print(f"⚡ Energy: {current_obs.energy}/100")
            print(f"\n🤖 Model Response:\n{response}")
            print(f"\n🎯 Parsed Action: {action.action_type.value}")
            if action.message_content:
                print(f"💬 Message: \"{action.message_content}\"")
        
        # Execute
        actions_dict = {
            sid: Action(sailor_id=sid, action_type=ActionType.WAIT)
            for sid in env.agents
        }
        actions_dict[sailor_id] = action
        
        try:
            obs_dict, rewards, dones, _, info = env.step(actions_dict)
            current_obs = obs_dict[sailor_id]
            reward = rewards[sailor_id]
            
            history.append((action, reward, current_obs))
            
            if verbose:
                print(f"💰 Reward: {reward:.2f}")
                print(f"📊 Ship: {current_obs.ship_progress.total_percentage}%")
            
            if dones[sailor_id]:
                print(f"\n🏁 Game over for {sailor_id}")
                break
                
        except Exception as e:
            print(f"❌ Error: {e}")
            break
    
    print("\n" + "=" * 80)
    print(f"\n📊 SUMMARY ({len(history)} turns completed):")
    total_reward = sum(r for _, r, _ in history)
    print(f"   Total Reward: {total_reward:.2f}")
    print(f"   Average Reward: {total_reward/len(history):.2f}" if history else "   No turns completed")
    print(f"   Final Ship Progress: {current_obs.ship_progress.total_percentage}%")
    print(f"   Final Energy: {current_obs.energy}/100")
    
    return history

# Run 5 turns
history = run_ai_turns(num_turns=5, sailor_id="Alice", verbose=True)


## 📊 Analyze AI Behavior

What did the AI do? Did it make sense?

In [ ]:
if history:
    print("🔍 AI BEHAVIOR ANALYSIS")
    print("=" * 80)
    
    # Count action types
    action_counts = {}
    for action, reward, obs in history:
        action_type = action.action_type.value
        action_counts[action_type] = action_counts.get(action_type, 0) + 1
    
    print("\n📈 Action Distribution:")
    for action_type, count in sorted(action_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"   {action_type}: {count} times")
    
    # Analyze rewards
    rewards = [r for _, r, _ in history]
    print(f"\n💰 Reward Analysis:")
    print(f"   Max Reward: {max(rewards):.2f}")
    print(f"   Min Reward: {min(rewards):.2f}")
    print(f"   Avg Reward: {sum(rewards)/len(rewards):.2f}")
    
    # Check for movement
    positions = [(obs.position.x, obs.position.y, obs.position.level) for _, _, obs in history]
    unique_positions = len(set(positions))
    print(f"\n🗺️ Movement Analysis:")
    print(f"   Unique Positions Visited: {unique_positions}/{len(history)}")
    print(f"   Explored: {unique_positions > 1}")
    
    # Check for resource gathering
    gathered_resources = sum(1 for action, _, _ in history if action.action_type == ActionType.GATHER_RESOURCE)
    print(f"\n🌲 Resource Gathering:")
    print(f"   Gather Attempts: {gathered_resources}")
    
    # Check energy management
    energies = [obs.energy for _, _, obs in history]
    print(f"\n⚡ Energy Management:")
    print(f"   Starting Energy: {energies[0] if energies else 'N/A'}")
    print(f"   Ending Energy: {energies[-1] if energies else 'N/A'}")
    print(f"   Net Change: {energies[-1] - energies[0] if energies else 'N/A'}")
    
    print("\n" + "=" * 80)
else:
    print("❌ No history to analyze")

## 🎓 Testing Different Scenarios

Test the AI in various game situations!

In [ ]:
# Reset environment with different seed for variety
print("🔄 Resetting environment with new seed...\n")
observations = env.reset(seed=999)

# Try with a different sailor
test_sailor = "Bob"
print(f"🧪 Testing with {test_sailor}...\n")

# Run 3 turns
history_bob = run_ai_turns(num_turns=3, sailor_id=test_sailor, verbose=True)

## 🔮 Future: Load Trained Model

After training, you can load your trained LoRA weights here:

```python
# Load trained adapter
from peft import PeftModel

model = PeftModel.from_pretrained(
    model,
    "outputs_marooned_rl/checkpoint-300",  # Path to your trained checkpoint
)

print("✅ Trained model loaded!")
```

Then run the same tests above to see if the trained model:
- ✅ Makes smarter moves
- ✅ Gathers resources more efficiently
- ✅ Builds the ship strategically
- ✅ Uses social deduction (lies as traitor, detects as colonist)
- ✅ Earns higher rewards

---

## ✅ Success Checklist

After running this notebook, you should see:

- [x] Environment loads successfully
- [x] Model generates responses
- [x] Actions are parsed correctly
- [x] Actions execute in environment
- [x] Rewards are calculated (Phase 4 system)
- [x] Multiple turns run without errors
- [x] Agent shows some coherent behavior

**Base model** (untrained) will likely:
- ❓ Make random or simple moves
- ❓ Not follow complex strategies
- ❓ Get low rewards

**After training**, the model should:
- ✅ Navigate purposefully toward resources
- ✅ Gather and deposit efficiently
- ✅ Coordinate ship building
- ✅ Use deception (if traitor) or detection (if colonist)
- ✅ Earn higher average rewards

---

## 🎯 Next Steps:

1. **Run this notebook** to test base model
2. **Train model** using `Train_Marooned_OpenEnv_RL.ipynb`
3. **Come back here** to test trained model
4. **Compare performance** - did training help?
5. **Iterate** - adjust rewards, prompts, training params

Good luck! 🏴‍☠️

---

## 📝 Summary: System Prompt vs User Prompt

### The Proper Two-Prompt Architecture:

**1. SYSTEM PROMPT** (Set ONCE at initialization):
```
Role: system
Content: Complete game rules, mechanics, objectives, win conditions, strategy tips

For Colonist:
- Game overview (5 sailors, rebuild ship, find traitor)
- Ship construction requirements
- Energy system, poison mechanics
- Detection strategies
- Win conditions

For Traitor:
- Game overview (same island, different goal)
- Sabotage tactics, deception techniques
- Poison strategy, special abilities
- Win conditions
```

**2. USER PROMPT** (Changes EVERY turn):
```
Role: user
Content: Current observation ONLY

- Day/Turn/Phase
- Current position, energy, backpack
- Spatial view (11×11 grid)
- Nearby resources, sailors
- Ship progress
- Team status
- Available actions
```

### Why This Is Better:

✅ **Token Efficiency:**
- System prompt: ~1,500 tokens (set once)
- User prompt: ~800 tokens (changes each turn)
- Old way: ~2,300 tokens every turn
- New way: 1,500 (once) + 800 (per turn) = massive savings

✅ **Cleaner Separation:**
- Game rules = System (doesn't change)
- Current state = User (updates constantly)

✅ **Better Training:**
- Model learns game rules are constant context
- Observations are dynamic input
- Clearer prompt structure

✅ **Role-Specific Context:**
- Colonists get colonist strategies
- Traitor gets sabotage tactics
- No wasted tokens on irrelevant info

---